In [1]:

# 꽉차있는 폭탄을 피하면서, 전체 타일을 뒤집는 게임입니다. 숫자는 그 주위에 차 있는 폭탄 수를 나타냅니다.


import sys
from math import floor
import pygame
from random import randint
from pygame.locals import QUIT,MOUSEBUTTONDOWN

WIDTH=20
HEIGHT= 15
SIZE = 50
EMPTY= 0
NUM_OF_BOMBS=20
BOMB=1
OPEN_COUNT= 0
CHECKED =[ [0 for _ in range(WIDTH)] for _ in range(HEIGHT)]
OPENED = 2

pygame.init()
SURFACE = pygame.display.set_mode((WIDTH*SIZE,HEIGHT*SIZE))
FPSCLOCK = pygame.time.Clock()
#######################################################
import cv2
import numpy as np
# --- OpenCV 동영상 저장 설정 ---
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('pygame_record.avi', fourcc, 15.0, (1000, 750))  # FPS 15
##########################################################
def num_of_bomb(field,x_pos,y_pos): # xpos와 ypos는 마우스클릭 위치가 아니라.  field 리스트에서 위치를 말하는거임
    # 주위에 있는 폭탄의 수를 반환한다.
    count = 0
    for yoffset in range(-1,2):
        for xoffset in range(-1,2):
            xpos,ypos= (x_pos+xoffset, y_pos+yoffset)
            if 0 <= xpos < WIDTH  and 0 <= ypos < HEIGHT and field[ypos][xpos] == BOMB:
                count += 1
    return count

def open_tile(field,x_pos,y_pos):   # xpos와 ypos는 마우스클릭 위치가 아니라.  field 리스트에서 위치를 말하는거임
    # 타일을 오픈
    global OPEN_COUNT
    if CHECKED[y_pos][x_pos]:  #이미 확인된 파일
        return                 # “이 함수, 여기서 끝. 이제 밖으로 나갈게.” CHECKED값이 True이면 
    
    CHECKED[y_pos][x_pos] = True  #함수의 무한반복 실행을 막기위함이다.

    for yoffset in range(-1,2):
        for xoffset in range(-1,2):
            xpos,ypos= (x_pos+xoffset, y_pos+yoffset)
            if 0 <= xpos < WIDTH  and 0 <= ypos < HEIGHT and field[ypos][xpos] == EMPTY:
                field[ypos][xpos] = OPENED
                OPEN_COUNT += 1

                count= num_of_bomb(field,xpos,ypos)
                if count ==0 and not(xpos==x_pos and ypos==y_pos):
                    open_tile(field,xpos,ypos)

def main():

    smallfont=  pygame.font.SysFont(None,36)
    largefont=  pygame.font.SysFont(None,72)
    message_clear= largefont.render("CLEARED!!", True,(0,255,255))
    message_over = largefont.render("Game Over!!", True,(0,255,255))

    message_rect = message_clear.get_rect()
    message_rect.center=(WIDTH*SIZE/2,HEIGHT*SIZE/2)

    field= [[EMPTY for xpos in range(WIDTH)] for ypos in range(HEIGHT)]
    game_over=False

    # 폭탄을 설치
    count = 0
    while count < NUM_OF_BOMBS:
        xpos,ypos= randint(0,WIDTH-1),randint(0,HEIGHT-1)
        if field[ypos][xpos] == EMPTY:
            field[ypos][xpos]= BOMB
            count +=1
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == MOUSEBUTTONDOWN and event.button==1:
                xpos,ypos= floor(event.pos[0]/SIZE),floor(event.pos[1]/SIZE)
                if field[ypos][xpos] == BOMB:
                    game_over= True
                else:
                    open_tile(field,xpos,ypos)

        # 그리기
        SURFACE.fill((10,0,0))
        for ypos in range(HEIGHT):
            for xpos in range(WIDTH):
                tile= field[ypos][xpos]
                rect= (xpos*SIZE,ypos*SIZE,SIZE,SIZE)

                if tile == EMPTY or tile==BOMB:
                    pygame.draw.rect(SURFACE,(192,192,192),rect)
                    
                    if game_over and tile ==BOMB:
                        pygame.draw.ellipse(SURFACE,(225,225,0),rect)
                elif tile==OPENED:
                    count = num_of_bomb(field,xpos,ypos)
                    if count > 0 :
                        num_image=smallfont.render(f"{count}",True,(255,255,0))
                        SURFACE.blit(num_image,(xpos*SIZE+10,ypos*SIZE+10))

               
        # 선그리기
        for i in range(0,WIDTH*SIZE,SIZE):
            pygame.draw.line(SURFACE,(96,96,96),(i,0),(i,HEIGHT*SIZE))

        for j in range(0,HEIGHT*SIZE,SIZE):
            pygame.draw.line(SURFACE,(96,96,96),(0,j),(WIDTH*SIZE,j))

        # 메세지 나타내기
        if OPEN_COUNT == WIDTH*HEIGHT  -  NUM_OF_BOMBS:
            SURFACE.blit(message_clear,message_rect.topleft)
        
        elif game_over:
            SURFACE.blit(message_over,message_rect.topleft)
            

        pygame.display.update()
        # --- 화면을 OpenCV로 저장 ---
        frame = pygame.surfarray.array3d(SURFACE)
        frame = np.transpose(frame, (1, 0, 2))  # 좌표계 변환
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        out.write(frame)
        FPSCLOCK.tick(10)

if __name__ =='__main__':
    main()

c:\Users\고창섭\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\고창섭\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


> 02 마인스위퍼  두번째 작성

꽉차 있는 폭탄을 피하면서 전체 타일을 뒤집는 게임입니다. 숫자는 그 주위에 차 있는 폭탄 수를 나타냅니다. 이런 게임도 구현할 수 있다는 것을 알기 위한 게임이다.

In [8]:
##mine_sweeper the second try##########

import sys
from random import randint
from math import floor  #실수를입력하면 그보다작은정수로반환해준다.
from pygame.locals import QUIT,MOUSEBUTTONDOWN
import pygame

WIDTH=20
HEIGHT= 15
SIZE = 50
EMPTY= 0  #아무것도 없음(아직열려있지 않다)
BOMB=1      #폭탄있음(아직열려있지않다)->열면즉시게임오버
OPENDED =2  #이미 열려있다.
OPEN_COUNT = 0      #열린타일의수
CHECKED=[ [0 for _ in range(WIDTH)] for _ in range(HEIGHT)] #타일의상태를이미확인했는지 기록하는 배열
NUM_OF_BOMBS=20
pygame.init()
SURFACE = pygame.display.set_mode((WIDTH*SIZE,HEIGHT*SIZE))
    #전역변수로 SURFACE를 설정한다. ()안에 ()로 숫자를 채우는거에 주의해야한다.
FPSCLOCK = pygame.time.Clock()
    #전역변수로 FPSCLOCK를 설정한다(프레임레이트조정용타이머)

bang_sound = pygame.mixer.Sound(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\bang.wav")   # 짧은 wav/mp3 파일
# 실행 hit_sound.play()

# pygame.mixer.init()       # 음악 로드
# pygame.mixer.music.load(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\airplane.wav")   # wav, mp3 둘 다 가능
# pygame.mixer.music.play(-1) # 반복 재생 ( -1 : 무한 반복 )
# pygame.mixer.music.set_volume(0.5) # 볼륨 조절 (0.0 ~ 1.0)


####시작준비화면####################################################################################
from pygame.locals import KEYDOWN,K_SPACE
pygame.display.set_caption("mine sweeper") # 글꼴 설정
font = pygame.font.SysFont(None, 72)
text = font.render("Start press SPACE!", True, (100,255,100))
text_rect = text.get_rect(center=(500,300))

# --- 1단계: 대기 화면 ---
waiting = True
while waiting:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            raise SystemExit
        if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
            waiting = False  # 스페이스키 눌리면 대기 종료
    SURFACE.fill((0,0,0))
    SURFACE.blit(text, text_rect)
    pygame.display.flip()
    FPSCLOCK.tick(30)
###################################################################################

def num_of_bomb(field,x_pos,y_pos): # 주위에 있는 폭탄의 수를 반환한다.
    count = 0
    for yoffset in range(-1,2):
        for xoffset in range(-1,2):  #총9칸을 검사해서
            xpos,ypos= (x_pos+xoffset, y_pos+yoffset) #검사할 칸으로 이동
            if 0 <= xpos < WIDTH  and 0 <= ypos < HEIGHT and field[ypos][xpos] == BOMB: #검사할칸이 보드내이고  AND 폭탄이면 카운트증가
                count += 1
    return count

def open_tile(field,x_pos,y_pos):
    global OPEN_COUNT
    if CHECKED[y_pos][x_pos]:  #이미 확인된 파일
        return                 # “이 함수, 여기서 끝. 이제 밖으로 나갈게.” CHECKED값이 True이면 
    
    CHECKED[y_pos][x_pos] = True  #함수의 무한반복 실행을 막기위함이다.

    for yoffset in range(-1,2):
        for xoffset in range(-1,2):
            xpos,ypos= (x_pos+xoffset, y_pos+yoffset)
            if 0 <= xpos < WIDTH  and 0 <= ypos < HEIGHT and field[ypos][xpos] == EMPTY:
                field[ypos][xpos] = OPENDED
                OPEN_COUNT += 1

                count= num_of_bomb(field,xpos,ypos)
                if count ==0 and not(xpos==x_pos and ypos==y_pos):
                    open_tile(field,xpos,ypos)
def main():
    smallfont=  pygame.font.SysFont(None,36)
    largefont=  pygame.font.SysFont("malgun.ttf",72)
    message_clear= largefont.render("CLEARED!!", True,(0,255,255))
    message_over = largefont.render("Game Over!!", True,(0,255,255))    
    message_rect = message_clear.get_rect()
    message_rect.center=(WIDTH*SIZE/2,HEIGHT*SIZE/2)
    # 렌더링할 기호
    symbol_text = "?"  # 예: 하트 기호
    # 텍스트를 Surface 객체로 변환
    text_surface = largefont.render(symbol_text, True, (255, 0, 0))

    game_over=False
    flag_list= []
    field= [[EMPTY for xpos in range(WIDTH)] for ypos in range(HEIGHT)]
    #폭탄을 설치
    count=0
    while count < NUM_OF_BOMBS:
        xpos,ypos= randint(0,WIDTH-1),randint(0,HEIGHT-1)
        if field[ypos][xpos] == EMPTY:
            field[ypos][xpos]= BOMB
            count +=1


    while True:

        for event   in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == MOUSEBUTTONDOWN and event.button==1:
                xpos,ypos= floor(event.pos[0]/SIZE),floor(event.pos[1]/SIZE)
                if field[ypos][xpos] == BOMB:
                    bang_sound.play()
                    game_over= True
                else:
                    open_tile(field,xpos,ypos)
            if event.type == MOUSEBUTTONDOWN and event.button==3:
                f_xpos,f_ypos= floor(event.pos[0]/SIZE),floor(event.pos[1]/SIZE)
                flag_list.append((f_xpos*SIZE+10,f_ypos*SIZE+10))
        #그리기
        SURFACE.fill((255,255,255))
        for ypos in range(HEIGHT):
            for xpos in range(WIDTH):
                tile= field[ypos][xpos]
                rect= (xpos*SIZE,ypos*SIZE,SIZE,SIZE)
                if tile == EMPTY or tile==BOMB:
                    pygame.draw.rect(SURFACE,(192,192,192),rect)
                    if game_over and tile ==BOMB:
                        pygame.draw.ellipse(SURFACE,(225,225,0),rect)
                elif tile==OPENDED:
                    count = num_of_bomb(field,xpos,ypos)
                    if count > 0 :
                        num_image=smallfont.render(f"{count}",True,(255,255,0))
                        SURFACE.blit(num_image,(xpos*SIZE+10,ypos*SIZE+10))
        for f in flag_list:    # 화면의 원하는 위치 (x, y)에 Surface 배치
            SURFACE.blit(text_surface, f)
                    
        #선그리기
        for i in range(0,WIDTH*SIZE,SIZE):
            pygame.draw.line(SURFACE,(96,96,96),(i,0),(i,HEIGHT*SIZE))
        for j in range(0,HEIGHT*SIZE,SIZE):
            pygame.draw.line(SURFACE,(96,96,96),(0,j),(WIDTH*SIZE,j))
        #메세지나타내기
        if OPEN_COUNT == WIDTH*HEIGHT  -  NUM_OF_BOMBS:
            SURFACE.blit(message_clear,message_rect.topleft)
        elif game_over:
            SURFACE.blit(message_over,message_rect.topleft)
        pygame.display.update()
        FPSCLOCK.tick(15)
if __name__=='__main__':
    main()

SystemExit: 